In [ ]:
import sys, os
from time import time
if '../' not in sys.path:
    sys.path.insert(0, '../')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from modules.paths import Experiment
from modules.background import LinearCorrection, GLMCorrection, StackCorrector
from modules.io import IO

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Load paths to each set of experiments
loc = '/Volumes/bernasek_t5/pnt_manuscript/microscopy/yan_clones/'
#loc = '/Users/Sebi/Desktop/clones/'
control_path = os.path.join(loc, 'control/slices/w-eyFLP_PntGFP_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A')
mutant_path = os.path.join(loc, 'perturbation/slices/w-eyFLP_PntGFP_YanE833_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A')
control_exp = Experiment(control_path)
perturbation_exp = Experiment(mutant_path)
experiments = dict(control=control_exp, perturbation=perturbation_exp)

In [ ]:
""" 
P-25-2 had to change percentile to 99.99 

P-46-1 is a good example
"""

# correct all stacks

In [ ]:
exp = experiments['perturbation']
for stack in exp:
    try:
        corr = StackCorrector(stack, mode='glm', niters=20, resample=True, resample_cutoff=99.999)
        corr.save()
    except:
        print('FAIL: Disc {:s}'.format(stack.disc_name))

# correct individual stack

In [ ]:
exp = experiments['perturbation']
stack = exp.discs[53].load_stack()

In [ ]:
corr = StackCorrector(stack, mode='glm', niters=20, resample=True, resample_cutoff=99.999)

In [ ]:
corr.show()

In [ ]:
corr.save()

In [ ]:
layer_id = 3
corr.correct(layer_id=layer_id, 
             mode='glm',
             niters=10, 
             resample=True,
             resample_cutoff=99.999)
correction = corr.corrections[layer_id]
correction.show_correction(furrow_only=False)
correction.show_fit()
fig = correction.show_pixel_distributions()
fig = correction.show_mask()